In [144]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction import text  
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict


import sqlite3
cnx = sqlite3.connect('db.sqlite3')

def is_spam(EmailText):
   
    dataframe = pd.read_sql_query("SELECT * FROM application_data_set",cnx)
    df = dataframe.copy()
    
    X = df['EmailText']
    Y = df['Label']
    X_train,X_test,Y_train,Y_test= train_test_split(X,Y, test_size=0.2, random_state=42)


    vectorizer = text.CountVectorizer(stop_words="english")
    counts = vectorizer.fit_transform(X_train.values)
    spam_nb = MultinomialNB()
    spam_nb.fit(counts,Y_train.values)
    
    counts_test = vectorizer.transform(X_test.values)
    y_test_pred = predict_test_vect = spam_nb.predict(counts_test)
    
    cm = confusion_matrix(Y_test, y_test_pred) 
    print(Y_test.shape)
    print( len(y_test_pred))
    print(cm)

    test_vect = vectorizer.transform(np.array([EmailText]))
    predict_test_vect = spam_nb.predict(test_vect)
    new_row = {'Label':predict_test_vect[0],'EmailText':EmailText,'id' :df.shape[0]+1}
    dataframe = dataframe.append(new_row,ignore_index=True)
    dataframe.to_sql(name='application_data_set',con=cnx,if_exists = 'replace',index=False)
    
    return predict_test_vect[0] == 'spam'


In [145]:
print(is_spam("This is to inform you on the release of the EL-GORDO SWEEPSTAKE LOTTERY PROGRAM held on the 29th OF APRIL 2005. Your name is attached to ticket number 025-11-464-992-750 with serial number 2113-05 drew the lucky numbers 13-15-16-21-34-36 which consequently won the lottery in the 3rd category."))

(1132,)
1132
[[956   6]
 [ 15 155]]
True


In [125]:

df = pd.read_sql_query("SELECT * FROM application_data_set",cnx)
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)
X_train = train_set['EmailText']
Y_train = train_set['Label']
X_test = test_set['EmailText']
Y_test = test_set['Label']

vectorizer = text.CountVectorizer(stop_words="english")
counts = vectorizer.fit_transform(X_train.values)
spam_nb = MultinomialNB()
spam_nb.fit(counts,Y_train.values)


labels = ['ham', 'spam']    